In [25]:
import pandas as pd
from math import sin, cos, sqrt, atan2, tan
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

def block(file, string, pattern): 
    
    should_print = False
    for line in string:
        if line.startswith(pattern):
               should_print = True
        elif not line.startswith(' '):
               should_print = False
        if should_print:
               file.write(line)
                

with open("C:\\Users\\r.piliai\\Downloads\\123\\brdm1990.rnx") as fh:
    
    string = fh.readlines()
    
f = open("C:\\Users\\r.piliai\\Downloads\\123\\data1_test.txt", "w")


a = ["%.2d" % i for i in range(37)]

c = ["E" + str(i) for i in a]
for pattern in c:
    block(f, string, pattern)
    
f.close()

In [26]:


f1 = open('C:\\Users\\r.piliai\\Downloads\\123\\data1_test.txt', 'r')
f2 = open('C:\\Users\\r.piliai\\Downloads\\123\\file2.txt', 'w')
for line in f1:
    f2.write(line.replace('-', ' -'))
f1.close()
f2.close()

f1 = open('C:\\Users\\r.piliai\\Downloads\\123\\file2.txt', 'r')
f2 = open('C:\\Users\\r.piliai\\Downloads\\123\\good.txt', 'w')
for line in f1:
    f2.write(line.replace('e -', 'e-'))
f1.close()
f2.close()


In [27]:
with open('good.txt') as f:
    res = []
    for line in f:
        if line.startswith('E'):
            res.append([])
        res[-1].extend(line.split())

f = open("data2_test.txt", "w")
f.write(str(res))
f.close()

In [28]:
f4 = pd.DataFrame(res)

In [29]:
f5 = f4.drop_duplicates(subset = f4.iloc[:,0:6])
    

In [30]:
f4.iloc[:,0:6]

,0,1,2,3,4,5
0,E01,2019,07,18,00,10
1,E01,2019,07,18,00,00
2,E01,2019,07,18,00,00
3,E01,2019,07,18,00,10
4,E01,2019,07,18,00,10
...,...,...,...,...,...,...
7212,E36,2019,07,18,23,00
7213,E36,2019,07,18,23,10
7214,E36,2019,07,18,23,20
7215,E36,2019,07,18,23,30


In [31]:
f5 = f5.reset_index()

In [32]:
f5 = f5.drop(f5.columns[0], axis=1)

In [33]:

ff = f5

In [34]:
ff.iloc[0,0]

'E01'

In [35]:
ff.columns = ['prn','t1','t2','t3','t4','t5','t6','a0','a1','a2','IODE','Crs','dn','M0','Cuc','e','Cus','sqrta','t0e','Cic','omega0','Cis','i0','Crc','omega','pr_omega','IDOT',
              'kod_L2','GPSweek','SV','SV_H','TGD','IDOC','ttc']


ff = ff.apply(pd.to_numeric, errors='ignore')

In [36]:
def modp(n, m):
    return n - np.abs(m) * np.floor(n / np.abs(m))

In [41]:
rr = pd.DataFrame()
D = 4
rr['t'] = D * 24 * 60 * 60 + ff['t3'] * 60 * 60 + ff['t4'] * 60 + ff['t5']

omega_e = 0.000072921151467
mi= 3.986005 * 10**14
pi= 3.1415926535898
c= 299792458
rr['t0c'] = rr['t']
rr['dt'] = ff['a0'] + ff['a1'] * (rr['t'] - rr['t0c']) + ff['a2'] * ((rr['t']-rr['t0c'])**2)
rr['tk'] = rr['t'] - ff['t0e']
rr['a'] = np.power(ff['sqrta'],2)
rr['n0'] = np.sqrt(mi/(rr['a']**3))
rr['n'] = rr['n0'] + ff['dn']
rr['Mk'] = modp((ff['M0'] + (rr['n'] * rr['tk'])), 2 * pi)

E1 = []
for i in range(len(ff)):
    Ek = 1
    E = float(rr['Mk'][i]) + (float(ff['e'][i]) * sin(Ek))
    while abs(Ek-E) > (10**(-14)):
        Ek = E
        E = modp((float(rr['Mk'][i]) + (float(ff['e'][i]) * sin(Ek))), 2 * pi)
    E1.append(E)
    
rr['Ek'] = E1

v=[]
for i in range(len(ff)):
    
    vv = atan2(np.sqrt(1 - ff['e'][i]**2) * np.sin(rr['Ek'][i]), np.cos((rr['Ek'][i]) - ff['e'][i]))
    v.append(vv)
rr['v'] = v
rr['u'] = modp((ff['omega'] + rr['v']), 2 * pi)
rr['duk'] = ff['Cus'] * np.sin(2 * rr['u']) + ff['Cuc'] * np.cos(2 * rr['u'])
rr['drk'] = ff['Crs'] * np.sin(2 * rr['u']) + ff['Crc'] * np.cos(2 * rr['u'])
rr['dik'] = ff['Cis'] * np.sin(2 * rr['u']) + ff['Cic'] * np.cos(2 * rr['u']) + ff['IDOT'] * rr['tk']
rr['uk'] = rr['u'] + rr['duk']
rr['rk'] = rr['a'] * (1 - ff['e'] * np.cos(rr['Ek'])) + rr['drk']
rr['ik'] = ff['i0'] + rr['dik']
rr['omegak'] = modp((ff['omega0'] + (ff['pr_omega'] - omega_e) * rr['tk'] - omega_e * ff['t0e']),  2 * pi)
rr['ksi'] = rr['rk'] * np.cos(rr['uk'])
rr['eta'] = rr['rk'] * np.sin(rr['uk'])
rr['X'] = (rr['ksi'] * np.cos(rr['omegak']) - rr['eta'] * np.cos(rr['ik']) * np.sin(rr['omegak']))/1000
rr['Y'] = (rr['ksi'] * np.sin(rr['omegak']) + rr['eta'] * np.cos(rr['ik']) * np.cos(rr['omegak']))/1000
rr['Z'] = (rr['eta'] * np.sin(rr['ik']))/1000
# rr['EM'] = 1/(1 - ff['e'] * cos(rr['Ek']))
# rr['pr_v'] = sqrt((1 + ff['e'])/(1 - ff['e'])) * (1/((cos(rr['Ek']/2))**2)) * (1/(1 + ((tan(rr['v']/2))**2))) * rr['EM'] * rr['n']
# rr['pr_u'] =rr['pr_v'] + (2 * rr['pr_v']) * (ff['Cus'] * cos(2 * rr['u']) - ff['Cuc'] * sin(2 * rr['u']))
# rr['pr_omega_m'] = ff['pr_omega'] - omega_e
# rr['pr_i'] = ff['IDOT'] + (2 * rr['pr_v']) * (ff['Cis'] * cos(2 * rr['u']) - ff['Cic'] * sin(2 * rr['u']))
# rr['pr_r'] = a * ff['e'] * sin(rr['Ek']) * rr['EM'] * rr['n'] + (2 * rr['pr_v']) * (ff['Crs'] * cos(2 * rr['u']) - ff['Crc'] * sin(2 * rr['u']))
# rr['pr_ksi'] = rr['pr_r'] * cos(rr['uk']) - rr['rk'] * sin(rr['uk']) * rr['pr_u']
# rr['pr_eta'] = rr['pr_r'] * sin(rr['uk']) + rr['rk'] * cos(rr['uk']) * rr['pr_u']
# rr['pr_X'] = cos(rr['omegak']) * rr['pr_ksi'] - cos(rr['ik']) * sin(rr['omegak']) * rr['pr_eta'] - rr['ksi'] * sin(rr['omegak']) * rr['pr_omega_m'] - rr['eta'] * cos(rr['ik']) * cos(rr['omegak']) * rr['pr_omega_m'] + rr['eta'] * sin(rr['ik']) * sin(rr['omegak']) * rr['pr_i']
# rr['pr_Y'] = sin(rr['omegak']) * rr['pr_ksi'] + cos(rr['ik']) * cos(rr['omegak']) * rr['pr_eta'] + rr['ksi'] * cos(rr['omegak']) * rr['pr_omega_m'] - rr['eta'] * cos(rr['ik']) * sin(rr['omegak']) * rr['pr_omega_m'] - rr['eta'] * sin(rr['ik']) * cos(rr['megak']) * rr['pr_i']
# rr['pr_Z'] = sin(rr['ik']) * rr['pr_eta'] + rr['eta'] * cos(rr['ik']) * rr['pr_i']
# rr['dt_rel'] = (-2) * (rr['X'] * rr['pr_X'] + rr['Y'] * rr['pr_Y'] + rr['Z'] * rr['pr_Z'])/(c**2)
# rr['dt_prim'] = (rr['dt'] + rr['dt_rel']) * 1000000

In [20]:
def XYZ(m, data):
    
    TT = [f4.iloc[1,m], f4.iloc[1,m], f4.iloc[1,m], f4.iloc[1,m], f4.iloc[1,m], f4.iloc[1,m]]
    T = [float(i) for i in TT]
    D = float(f4.iloc[1,m])
    t = D * 24 * 60 * 60 + T[3] * 60 * 60 + T[4] * 60 + T[5]

    omega_e = 0.000072921151467
    mi= 3.986005 * 10**14
    pi= 3.1415926535898
    c= 299792458


    PRN = f4.iloc[0,m]
    Toc = T
    a0 = float(f4.iloc[7,m])
    a1 = float(f4.iloc[8,m])
    a2 = float(f4.iloc[9,m])
    IODE = float(f4.iloc[10,m])
    Crs = float(f4.iloc[11,m])
    dn = float(f4.iloc[12,m])
    M0 = float(f4.iloc[13,m])
    Cuc = float(f4.iloc[14,m])
    e = float(f4.iloc[15,m])
    Cus = float(f4.iloc[16,m])
    sqrta = float(f4.iloc[17,m])
    t0e = float(f4.iloc[18,m])
    Cic = float(f4.iloc[19,m])
    omega0 = float(f4.iloc[20,m])
    Cis = float(f4.iloc[21,m])
    i0 = float(f4.iloc[22,m])
    Crc = float(f4.iloc[23,m])
    omega = float(f4.iloc[24,m])
    pr_omega = float(f4.iloc[25,m])
    IDOT = float(f4.iloc[26,m])
    # Kod_L2 = float(f4[27][m])
    # GPSweek = float(f4[28][m])
    # L2_P = float(f4[29][m])
    # SV = float(f4[30][m])
    # SV_H = float(f4[31][m])
    # TGD = float(f4[32][m])
    # IDOC = float(f4[33][m])
    #ttc = float(res[n][34])
    t0c = t
    # Wzory
    dt = a0 + a1 * (t - t0c) + a2 * ((t-t0c)**2)
    tk = t - t0e
    a = sqrta**2
    n0 = sqrt(mi/(a**3))
    n = n0 + dn
    Mk = modp((M0 + n * tk), 2 * pi)
    Ek = 1
    E = Mk + (e * sin(Ek))
    while abs(Ek-E) > (10**(-14)):
        Ek = E
        E = modp((Mk + (e * sin(Ek))), 2 * pi)
    v = atan2(sqrt(1 - e**2) * sin(Ek), cos(Ek) - e)
    u = modp((omega + v), 2 * pi)
    duk = Cus * sin(2 * u) + Cuc * cos(2 * u)
    drk = Crs * sin(2 * u) + Crc * cos(2 * u)
    dik = Cis * sin(2 * u) + Cic * cos(2 * u) + IDOT * tk
    uk = u + duk
    rk = a * (1 - e * cos(Ek)) + drk
    ik = i0 + dik
    omegak = modp((omega0 + (pr_omega - omega_e) * tk - omega_e * t0e),  2 * pi)
    ksi = rk * cos(uk)
    eta = rk * sin(uk)
    X = ksi * cos(omegak) - eta * cos(ik) * sin(omegak)
    Y = ksi * sin(omegak) + eta * cos(ik) * cos(omegak)
    Z = eta * sin(ik)
    X = X/1000
    Y = Y/1000
    Z = Z/1000
    EM = 1/(1 - e * cos(Ek))
    pr_v = sqrt((1 + e)/(1 - e)) * (1/((cos(Ek/2))**2)) * (1/(1 + ((tan(v/2))**2))) * EM * n
    pr_u = pr_v + (2 * pr_v) * (Cus * cos(2 * u) - Cuc * sin(2 * u))
    pr_omega_m = pr_omega - omega_e
    pr_i = IDOT + (2 * pr_v) * (Cis * cos(2 * u) - Cic * sin(2 * u))
    pr_r = a * e * sin(Ek) * EM * n + (2 * pr_v) * (Crs * cos(2 * u) - Crc * sin(2 * u))
    pr_ksi = pr_r * cos(uk) - rk * sin(uk) * pr_u
    pr_eta = pr_r * sin(uk) + rk * cos(uk) * pr_u
    pr_X = cos(omegak) * pr_ksi - cos(ik) * sin(omegak) * pr_eta - ksi * sin(omegak) * pr_omega_m - eta * cos(ik) * cos(omegak) * pr_omega_m + eta * sin(ik) * sin(omegak) * pr_i
    pr_Y = sin(omegak) * pr_ksi + cos(ik) * cos(omegak) * pr_eta + ksi * cos(omegak) * pr_omega_m - eta * cos(ik) * sin(omegak) * pr_omega_m - eta * sin(ik) * cos(omegak) * pr_i
    pr_Z = sin(ik) * pr_eta + eta * cos(ik) * pr_i
    dt_rel = (-2) * (X * pr_X + Y * pr_Y + Z * pr_Z)/(c**2)
    dt_prim = dt + dt_rel
    dt_prim = dt_prim * 1000000
    data.append(PRN)
    data.append(T)
    data.append(X)
    data.append(Y)
    data.append(Z)
    data.append(pr_X)
    data.append(pr_Y)
    data.append(pr_Z)
    data.append(dt_prim)
    #print(PRN, T, X, Y, Z, dt_prim)

In [23]:
data = []
for m in range(2,len(f4)):
    dane = []
    
    XYZ(m, data)
    dane.append(data)
    #print(data)
#dane.pop(0)
#print(dane)

KeyboardInterrupt: 

In [40]:

print(len(data))

306


In [ ]:
def block1(file, string, pattern): 
    
    should_print = False
    for line in string:
        if line.startswith(pattern):
                should_print = True
        elif not line.startswith('P'):
                should_print = False
        if should_print:
                file.write(line)
                        
with open(files[o][2]) as fb:
    
    string = fb.readlines()
                
f = open("example1.txt", "w")

pattern = '*  2019'

block1(f, string, pattern)
    
f.close()

fn = 'example1.txt'
output = open(fn).readlines()
f = open(fn, 'w')
f.writelines([item for item in output if 'PG' not in item])
f.close()
fn = 'example1.txt'
output = open(fn).readlines()
f = open(fn, 'w')
f.writelines([item for item in output if 'PR' not in item])
f.close()
fn = 'example1.txt'
output = open(fn).readlines()
f = open(fn, 'w')
f.writelines([item for item in output if 'PJ' not in item])
f.close()
fn = 'example1.txt'
output = open(fn).readlines()
f = open(fn, 'w')
f.writelines([item for item in output if 'PC' not in item])
f.close()

with open("example1.txt") as f:
    newText=f.read().replace('PE', 'E')

with open("example2.txt", "w") as f:
    f.write(newText)

example = []
base_val = ''
with open("example2.txt") as infile:
    for line in infile:
        line = line.strip()
        if line:  #Check if line is empty
            if line.startswith("*"):
                base_val = line.lstrip("*").strip().split()   #Get Date line
            else:
                example.append(base_val + line.split())        #Form required list
#print(example)